In [1]:
import cv2
import numpy as np

In [26]:
image = cv2.imread('shape.jpg')
image_grey = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
image_edge = cv2.Canny(image_grey,30,200)

In [27]:
_, contours, _ = cv2.findContours(image_edge, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
cv2.imshow('contour',image_edge)
cv2.waitKey()
cv2.destroyAllWindows()

In [29]:
cv2.drawContours(image,contours,-1,(255,0,0),3)
cv2.imshow('all contours',image)
cv2.waitKey()
cv2.destroyAllWindows()

### Sort

In [2]:
image = cv2.imread('shape.jpg')
image_blank = np.zeros((image.shape[0],image.shape[1],3))
image_original = image
image_grey = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
image_edge = cv2.Canny(image_grey,50,200)
_, contours, _ = cv2.findContours(image_edge.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)


In [3]:
def get_contours_ares(cnts):
    cnt_areas = []
    for cnt in cnts:
        cnt_areas.append(cv2.contourArea(cnt))
    return cnt_areas

In [4]:
print(get_contours_ares(contours))

[765402.0, 597328.0, 520390.5, 820834.5, 757505.5, 0.5]


In [5]:
contours_sorted = sorted(contours, key = cv2.contourArea, reverse = True)
print(get_contours_ares(contours_sorted))

[820834.5, 765402.0, 757505.5, 597328.0, 520390.5, 0.5]


In [6]:
for c in contours_sorted:
    cv2.drawContours(image_original,[c],-1,(0,0,255),3)
    cv2.waitKey(0)
    cv2.imshow('show',image_original)
    
cv2.waitKey()
cv2.destroyAllWindows()

### by x-axis 

In [7]:
def get_center_x(cnt):
    M = cv2.moments(cnt)
    return int(M['m10']/M['m00'])

In [11]:
def draw_center_label(image,cnt):
    M = cv2.moments(cnt)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    
    cv2.circle(image,(cx,cy),10,(0,0,255),-1)
    return image

In [12]:
image =cv2.imread('shape.jpg')
image_original = image.copy()

In [13]:
for cnt in contours:
    draw_center_label(image,cnt)

In [15]:
cv2.imshow('center dots',image)
cv2.waitKey()
cv2.destroyAllWindows()

In [16]:
counters_left_to_right = sorted(contours, key = get_center_x, reverse = True)

In [18]:
for i,cnt in enumerate(contours):
    cv2.drawContours(image,[cnt],-1,(0,0,255),3)
    M = cv2.moments(cnt)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    cv2.putText(image,str(i+1),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,2,(0,0,255),2)
    cv2.imshow('left to right',image)
    cv2.waitKey(0)
    x,y,w,h = cv2.boundingRect(cnt)
    
    image_crop = image[y:y+h,x:x+w]
    image_name = "imagecrap"+str(i)+".jpg"
    cv2.imwrite(image_name,image_crop)

### approxiamation

In [1]:
import cv2
import numpy as np

In [2]:
image = cv2.imread('shape1.jpg')
image_original = image.copy()

In [3]:
image_grey = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
image_edge = cv2.Canny(image_grey,50,200)
_, contours, _ = cv2.findContours(image_edge.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)

In [4]:
for c in contours:
    x,y,w,h = cv2.boundingRect(c)
    cv2.rectangle(image_original,(x,y),(x+w,y+h),(0,0,255),2)
    cv2.imshow('bound',image_original)
    cv2.waitKey()
    
cv2.waitKey()
 
for c in contours:
    accuracy = 0.01* cv2.arcLength(c,True)
    approx = cv2.approxPolyDP(c,accuracy, True)
    cv2.drawContours(image,[approx],0,(0,255,0),2)
    cv2.imshow('approx',image)
    cv2.waitKey()

cv2.destroyAllWindows()


### shape match

In [1]:
import cv2
import numpy as np

In [2]:
template = cv2.imread('imagecrap1.jpg')
template_grey = cv2.cvtColor(template,cv2.COLOR_BGR2GRAY)
target = cv2.imread('shape.jpg')
target_grey = cv2.cvtColor(target,cv2.COLOR_BGR2GRAY)

In [3]:
ret, threshold_temp = cv2.threshold(template_grey,127,255,0)
ret, threshold_target = cv2.threshold(target_grey, 127,255,0)

In [4]:
_,contours_temp,_ = cv2.findContours(threshold_temp,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
_,contours_target,_ = cv2.findContours(threshold_target,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)

In [19]:
contour_match = sorted(contours_temp,key = cv2.contourArea,reverse = True)[0]

cv2.drawContours(template,[contour_match],0,[255,0,0],2)
cv2.imshow('test',template)

cv2.waitKey()
cv2.destroyAllWindows()

In [20]:
contour_get = []
for c in contours_target:
    match_score = cv2.matchShapes(contour_match,c,1,0.0)
    if match_score<0.15:
        contour_get.append(c)
        print(match_score)

cv2.drawContours(target,contour_get,-1,[255,0,0],2)
cv2.imshow('output',target)
cv2.waitKey()
cv2.destroyAllWindows()
    


0.07354106828716106
0.013083300070283244
0.09907948266423805


### line-detect

In [1]:
import cv2
import numpy as np

In [2]:
image = cv2.imread('lines.jpg')
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
canny = cv2.Canny(gray,100,170)

In [3]:
lines = cv2.HoughLines(canny,1,np.pi/180,200)

In [22]:
lines[:,0,:]

array([[512.       ,   1.5707964],
       [138.       ,   1.5707964],
       [386.       ,   1.5707964],
       [142.       ,   1.5707964],
       [390.       ,   1.5707964],
       [447.       ,   1.5707964],
       [263.       ,   1.5707964],
       [200.       ,   1.5707964],
       [ 81.       ,   1.5707964],
       [325.       ,   1.5707964],
       [260.       ,   1.5707964],
       [ 76.       ,   1.5707964],
       [509.       ,   1.5707964],
       [451.       ,   1.5707964],
       [203.       ,   1.5707964],
       [329.       ,   1.5707964],
       [ 15.       ,   1.5707964],
       [152.       ,   0.       ],
       [212.       ,   0.       ],
       [580.       ,   0.       ],
       [699.       ,   0.       ],
       [761.       ,   0.       ],
       [332.       ,   0.       ],
       [516.       ,   0.       ],
       [702.       ,   0.       ],
       [ 87.       ,   0.       ],
       [ 90.       ,   0.       ],
       [271.       ,   0.       ],
       [639.       ,

In [15]:
for line in lines:
    rho = line[0][0]
    theta = line[0][1]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))
    
    cv2.line(image, (x1,y1),(x2,y2),(0,0,255),2)
    
cv2.imshow('line',image)
cv2.waitKey()
cv2.destroyAllWindows()

In [28]:
image = cv2.imread('lines.jpg')
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
canny = cv2.Canny(gray,100,170)
lines = cv2.HoughLinesP(canny, 1, np.pi/180,100,100,10)
for x1,y1,x2,y2 in lines[:,0,:]:
    cv2.line(image,(x1,y1),(x2,y2),(0,0,255),2)
    
cv2.imshow('p line',image)
cv2.waitKey()
cv2.destroyAllWindows()

### Blob detection

In [34]:
image = cv2.imread('emoji.jpg',cv2.IMREAD_GRAYSCALE)
detector = cv2.SimpleBlobDetector_create()
keypoints = detector.detect(image)

final = cv2.drawKeypoints(image,keypoints,np.array([]),(0,0,255),cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

cv2.imshow('final',final)
cv2.waitKey()
cv2.destroyAllWindows()